# Samay Lung Sound – ETL & Data Quality Report

**Autor:** _<Tu Nombre>_

Este notebook acompaña la entrega del pipeline. Lee las salidas de `./cleaned_data/`, resume calidad y corre consultas SQL.

## 0. Quickstart (reproducir)
```bash
python -m venv .venv
# Windows: .\.venv\Scripts\Activate.ps1
pip install -r requirements.txt
python -m src.cli --input "./raw_ingest" --output "./cleaned_data"
```

In [1]:
import os
os.chdir(r"C:\Users\lvarg\OneDrive\Escritorio\pipeline\samay-lung-pipeline")
print("CWD fijado a:", os.getcwd())


CWD fijado a: C:\Users\lvarg\OneDrive\Escritorio\pipeline\samay-lung-pipeline


## 1. Cargar salidas

In [2]:
import pandas as pd
from pathlib import Path
base = Path('./cleaned_data')
audio = pd.read_csv(base/'audio_recordings.csv')
patients = pd.read_csv(base/'patient_demographics.csv')
dq = pd.read_csv(base/'data_quality_report.csv')
audio.head(3), patients.head(3), dq.head(3)

(                               recording_id patient_id  \
 0  fd398df9d9ef0cdabc77f9f6c2348412c428feaa      P9064   
 1  ad609d7adc96bfbd0208668ce22676bed9e70589      P4951   
 2  ad99c5742abc36afd3589a1d9049c2747e67bf48      P4511   
 
                        timestamp  duration_seconds  sample_rate  bit_depth  \
 0  2025-03-08 00:00:00.000000000               NaN          NaN        NaN   
 1  2025-08-15 00:00:00.000000000               NaN          NaN        NaN   
 2  2024-10-02 00:00:00.000000000               NaN          NaN        NaN   
 
    filter_mode    recording_location                file_path      diagnosis  \
 0          NaN  Anterior_Right_Upper   01-06-2025_P9064_8.wav           COPD   
 1          NaN  Posterior_Left_Lower  01-08-2025_P4951_46.wav  Heart_Failure   
 2          NaN  Posterior_Left_Upper  02-10-2024_P4511_35.wav  Lung_Fibrosis   
 
                          hospital_site                          source_name  
 0  Samay Data Engineer -Technical test

## 2. Métricas básicas

In [ ]:
print('Registros de audio:', len(audio))
print('Pacientes (no nulos):', audio['patient_id'].dropna().nunique())
print('\nTop issues:')
print(dq.groupby(['severity','issue_type']).size().reset_index(name='count').sort_values(['severity','count'], ascending=[True, False]).head(10).to_string(index=False))

## 3. Visualizaciones

In [ ]:
import matplotlib.pyplot as plt
sr_counts = (audio['sample_rate'].dropna().astype(int).value_counts().head(10))
plt.figure(); sr_counts.plot(kind='bar'); plt.title('Top 10 sample_rate (Hz)'); plt.xlabel('sample_rate'); plt.ylabel('count'); plt.tight_layout(); plt.show()

In [ ]:
dur = audio['duration_seconds'].dropna().astype(float)
dur_clip = dur[(dur>=0) & (dur<=120)]
plt.figure(); plt.hist(dur_clip, bins=20); plt.title('Distribución de duración (s)'); plt.xlabel('seconds'); plt.ylabel('count'); plt.tight_layout(); plt.show()

## 4. Consultas a SQLite

In [ ]:
import sqlite3
con = sqlite3.connect('./cleaned_data/samay_lung.db')
qs = [
  "SELECT COUNT(*) AS n_audio FROM audio_recordings",
  "SELECT COUNT(DISTINCT patient_id) AS n_patients FROM audio_recordings WHERE patient_id IS NOT NULL",
  "SELECT issue_type, severity, COUNT(*) c FROM data_quality_issues GROUP BY 1,2 ORDER BY c DESC LIMIT 10",
]
import pandas as pd
for q in qs:
    print('\nSQL>', q)
    print(pd.read_sql(q, con))
con.close()